### Shot-Detection

In [ ]:
# def detect_scenes(video_path, scene_list):
#     cap = cv2.VideoCapture(video_path)   
#     embeddings = []
#     metadatas = []
#     ids = []
#     with tqdm(total = len(scene_list), desc = "Processing frames") as pbar:
#         for i, scene in enumerate(scene_list):
#             start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
#             mid_time = (start_time + end_time) / 2
#             timestamps = [start_time, mid_time, end_time]
#             labels = ["initial", "middle", "final"]
            
#             for t, label in zip(timestamps, labels):
#                 cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
#                 ret, frame = cap.read()
                
#                 if ret:
#                     img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                     image = Image.fromarray(img_rgb)
#                     inputs = processor(images = image, return_tensors = 'pt').to(device)
#                     blip_input = blip_processor(images = image, return_tensors = 'pt').to(device)
#                     with torch.no_grad():
#                         outputs = model.get_image_features(inputs.pixel_values)
#                         blip_outputs = blip_model.generate(**blip_input,
#                                                         # max_length = 500,
#                                                         # min_length = 150,
#                                                         # no_repeat_ngram_size=2,
#                                                         # num_beams = 5,
#                                                         )
                    
#                     caption = blip_processor.decode(blip_outputs[0], skip_special_tokens=True)
#                     image_embedding = outputs.pooler_output
#                     image_embedding = image_embedding / image_embedding.norm(dim = -1, keepdim= True)
#                     image_embedding = image_embedding.squeeze(0).cpu().numpy().tolist()
#                     timestamp_sec = t*1000
#                     frame_id = f"{video_path}:{timestamp_sec}"
                
#                     ids.append(frame_id)
#                     embeddings.append(image_embedding)

#                     metadatas.append({
#                         "frame_idx": f"frame_no_{i}_{label}",
#                         "caption": caption,
#                         "timestamp_ms": timestamp_sec,
#                         "source_path": video_path
#                     })
#             pbar.update(1)
                
#     return embeddings, metadatas, ids

In [ ]:
# def detect_scenes(video_path, scene_list):
#     cap = cv2.VideoCapture(video_path)   
#     embeddings = []
#     metadatas = []
#     ids = []
#     with tqdm(total = len(scene_list), desc = "Processing frames") as pbar:
#         for i, scene in enumerate(scene_list):
#             start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
#             mid_time = (start_time + end_time) / 2
#             timestamps = mid_time
#             labels = "middle"
            
            
#             cap.set(cv2.CAP_PROP_POS_MSEC, timestamps * 1000)
#             ret, frame = cap.read()
            
#             if ret:
#                 img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 image = Image.fromarray(img_rgb)
#                 inputs = processor(images = image, return_tensors = 'pt').to(device)
#                 blip_input = blip_processor(images = image, return_tensors = 'pt').to(device)
#                 with torch.no_grad():
#                     outputs = model.get_image_features(inputs.pixel_values)
#                     blip_outputs = blip_model.generate(**blip_input,
#                                                     # max_length = 60,
#                                                     # min_length = 20,
#                                                     # no_repeat_ngram_size=2,
#                                                     # num_beams = 5,
#                                                     )
                
#                 caption = blip_processor.decode(blip_outputs[0], skip_special_tokens=True)
#                 image_embedding = outputs.pooler_output
#                 image_embedding = image_embedding / image_embedding.norm(dim = -1, keepdim= True)
#                 image_embedding = image_embedding.squeeze(0).cpu().numpy().tolist()
#                 timestamp_sec = timestamps*1000
#                 frame_id = f"{video_path}:{timestamp_sec}"
            
#                 ids.append(frame_id)
#                 embeddings.append(image_embedding)

#                 metadatas.append({
#                     "frame_idx": f"frame_no_{i}_{labels}",
#                     "caption": caption,
#                     "timestamp_ms": timestamp_sec,
#                     "source_path": video_path
#                 })
#             pbar.update(1)
                
#     return embeddings, metadatas, ids

In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/40.2 MB ? eta -:--:--
   -- ------------------------------------- 2.1/40.2 MB 16.3 MB/s eta 0:00:03
   ---- ----------------------------------- 4.7/40.2 MB 14.3 MB/s eta 0:00:03
   ------- -------------------------------- 7.6/40.2 MB 13.9 MB/s eta 0:00:03
   ---------- ----------------------------- 10.2/40.2 MB 13.8 MB/s eta 0:00:03
   ------------- -------------------------- 13.1/40.2 MB 13.7 MB/s eta 0:00:02
   --------------- ------------------------ 15.7/40.2 MB 13.6 MB/s eta 0:00:02
   ------------------ --------------------- 18.4/40.2 MB 13.6 MB/s eta 0:00:02
   --------------------- ------------------ 21.2/40.2 MB 13.6 MB/s eta 0:00:02
   ----------------------- ---------------- 23.9/40.2 MB 13.5 MB/s eta 0:00:02
   -------------------------- ------------- 26.5/40.2 MB 13.5 MB/s eta 0:00:02
   ---------------------------- ----------- 29.1/40.2 MB 13.5 MB/


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Python314\python.exe -m pip install --upgrade pip


In [1]:
import sys
print(sys.executable)


c:\Users\fareh\coding\video-rag\.venv\Scripts\python.exe


In [1]:
import cv2
from scenedetect import open_video, SceneManager
import scenedetect
from scenedetect.detectors import ContentDetector
import os
from PIL import Image
from transformers import CLIPProcessor, CLIPModel, BlipProcessor, BlipForConditionalGeneration
import torch
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32",use_safetensors=True).to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 398/398 [00:00<00:00, 1030.61it/s, Materializing param=visual_projection.weight]                                
CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
vision_model.embeddings.position_ids | UNEXPECTED |  | 
text_model.embeddings.position_ids   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
The image processor of type `CLIPImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may

In [2]:
def scene_split(video_path):
    print("--- Detecting shot boundaries with PySceneDetect ---")
    video = open_video(video_path)
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector())

    try:
        scene_manager.detect_scenes(video, show_progress=True)
        scene_list = scene_manager.get_scene_list()
    except Exception as e:
        print("Scene detection failed:", e)
        scene_list = []
    return scene_list

In [10]:
def generate_captions_in_batches(batch_of_frames, clip_model, clip_processor, blip_model, blip_processor, device="cuda"):
    clip_inputs = clip_processor(images=batch_of_frames, return_tensors="pt").to(device)
    blip_inputs = blip_processor(images = batch_of_frames, return_tensors = 'pt').to(device)
    with torch.no_grad():
        clip_outputs = clip_model.get_image_features(clip_inputs.pixel_values)
        clip_outputs = clip_outputs.pooler_output
        clip_embeddings = clip_outputs / clip_outputs.norm(p=2, dim=-1, keepdim=True)
        blip_output_ids = blip_model.generate(**blip_inputs,
                                        max_length = 300,
                                        min_length = 100,
                                        no_repeat_ngram_size=3,
                                        repetition_penalty=1.5,
                                        early_stopping=True,
                                        do_sample=False,
                                        num_beams = 3,
                                        )
        captions = blip_processor.batch_decode(blip_output_ids, skip_special_tokens=True)

   
            
    embeddings_list = clip_embeddings.cpu().numpy().tolist()
    del clip_inputs, blip_inputs, clip_outputs, blip_output_ids
    torch.cuda.empty_cache()        

    return captions, embeddings_list

In [4]:
def frame_listing(scene_list, video_path):
    cap = cv2.VideoCapture(video_path)   
    ids = []
    frame_PIL = []
    timestamps_list = []
    with tqdm(total = len(scene_list), desc="processing frames ") as pbar: 
        for i, scene in enumerate(scene_list):
            start_time, end_time = scene[0].get_seconds(), scene[1].get_seconds()
            mid_time = (start_time + end_time) / 2
            timestamps = [start_time, mid_time, end_time]
            labels = ["initial", "middle", "final"]
            
            for t, label in zip(timestamps, labels):
                cap.set(cv2.CAP_PROP_POS_MSEC, t * 1000)
                ret, frame = cap.read()
                
                if ret:
                    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image = Image.fromarray(img_rgb)
                    frame_PIL.append(image)
                    timestamps_list.append(t)
                frame_id = f"{video_path}:{t}"
                ids.append(frame_id)
            pbar.update(1)
    return frame_PIL, timestamps_list, ids

In [5]:
def batched_captioning(frame_list: list, batch_size: int):
    caption_list = []
    embedding_list = []
    
    with tqdm(total = int(len(frame_list)/ batch_size), desc = "Batched image captioning") as pbar:
        for i in range(0, len(frame_list),batch_size):
            batch = frame_list[i:i+batch_size]
            caption , embedding = generate_captions_in_batches(batch, 
                                                            clip_model= model,
                                                            clip_processor= processor,
                                                            blip_model= blip_model,
                                                            blip_processor= blip_processor)
            caption_list.extend(caption)
            embedding_list.extend(embedding)
            pbar.update(1)
    return caption_list, embedding_list           


In [6]:
video_path = "../This Integral Breaks Math.mp4"

In [7]:
scene_list = scene_split("../This Integral Breaks Math.mp4")

--- Detecting shot boundaries with PySceneDetect ---


  Detected: 48 | Progress: 100%|██████████| 7793/7793 [00:27<00:00, 286.10frames/s]


In [8]:
frame_PIL, timestamps_list, ids = frame_listing(scene_list= scene_list, video_path= video_path)

processing frames :   0%|          | 0/49 [00:00<?, ?it/s]

processing frames : 100%|██████████| 49/49 [00:07<00:00,  6.27it/s]


In [11]:
caption_list, embeddings_list = batched_captioning(frame_list= frame_PIL, batch_size=8)  #300,100 5

Batched image captioning: 19it [01:54,  6.05s/it]                        


In [30]:
len(caption_list), caption_list[:10]

(146,
 ['there is a man standing in front of a black background holding up two fingers and making the peace sign with one hand, while the other hand is pointing at the same direction on the other side of the other way with the other finger to the sign that says 3x - 1 - 4 - 1 / 1 - 2 - 5 - 3 - 1 0 - 1s - 1? - 1 % - - - 7 - - 11 - - 9 - - 6 - - 8 - - 10 -',
  "there is a man standing in front of a blackboard with the word 3 - 1 written on it and pointing his fingers at the same time as if he does not know what to do you are doing this trick or not to make the other way to solve the problem for the problem, i ' s 2 - 4 - 5 - 1 - 0 - 3 - 2 - 7 - 1 / 2 - 1? - 3 / 3 - 4 / 3 / 2 / 3x - 2 / 1",
  "this is an image of a person writing on a blackboard with a calculator in the foreground and a hand holding a pen in front of their left hand, pointing to the sign that says ' s x - x - 1x - i - 1 - 2 - 3 - 4 - 5 - 1 / 3 - - - 0 - - 1s - - o - - 6 - - 9 - - 7 - - 8 - - 10 - - 3c - - 11 - 1",
  "this

In [49]:
# emb, met, ids = detect_scenes(video_path="../This Integral Breaks Math.mp4", scene_list= scene_list)

In [7]:
import chromadb
client = chromadb.PersistentClient(path = "../db_path")
collection = client.get_or_create_collection("frame_collection")

In [8]:
collection.add(
    ids = ids,
    embeddings= emb,
    metadatas= met,
)

In [13]:
w[0]

(00:00:00.000 [frame=0, fps=30.000], 00:00:01.233 [frame=37, fps=30.000])

In [13]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText

processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")
model = AutoModelForImageTextToText.from_pretrained("google/gemma-3-4b-it")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    },
]
inputs = processor.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(processor.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-3-4b-it.
401 Client Error. (Request ID: Root=1-6988df00-0c02343335eec9b9385a4499;82209a26-925c-4ae7-b207-ab86528e1a8e)

Cannot access gated repo for url https://huggingface.co/google/gemma-3-4b-it/resolve/main/config.json.
Access to model google/gemma-3-4b-it is restricted. You must have access to it and be authenticated to access it. Please log in.